In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
from scipy.ndimage import rotate as rotate_image
from scipy.signal import argrelextrema
import seaborn as sns
import time 


# P1: OTSU Algorithm

In [ ]:
plt.rcParams['figure.figsize'] = (12, 12)
def otsu(img_gray:np.array):
    start = time.time()
    img_flat = img_gray.flatten()
    p_i = np.array([(img_flat==level).sum() for level in range(256)],dtype=np.float64)
    p_i /= img_flat.shape
    
    results = []
    
    for k in range(1,255):
        p_1 = 0.0
        m_1 = 0.0
        m_g = 0.0 
        
        for index in range(k+1):
            m_1 += index*p_i[index]
            p_1 += p_i[index]
            m_g += index*p_i[index]
        
        for index in range(k+1,256):
            m_g += index*p_i[index]
        
        if p_1 == 0 :
            results.append((m_g*p_1 - m_1)**2)  
            continue  
        
        results.append(
            ((m_g*p_1 - m_1)**2)/(p_1*(1-p_1))
        )
            
    
    results = np.array(results,dtype=np.int32)
    _max_argus = np.argwhere(results == np.amax(results)).flatten()
    tresh = _max_argus.mean()
    ret,mask = cv2.threshold(img_gray,tresh,255,cv2.THRESH_BINARY)
    tresh = round(tresh,2)
    
    final = cv2.bitwise_and(img_gray,mask)  
    end = round(time.time() - start,2) 
    
    plt.subplot(2,2,1)
    plt.imshow(img_gray,cmap="gray")
    plt.title("Input image")
    plt.axis("off")
    
    
    plt.subplot(2,2,2)
    plt.bar(list(range(256)),p_i,color="tab:cyan")
    plt.plot(results/sum(results),color="blue")
    plt.plot([tresh,tresh],[0,max(p_i)],color="red")
    #plt.axis("off")
    plt.title("OTSU algorithm behavior on different gray levels | Threshold:{0} | Time:{1}".format(tresh,end))
    
    
    plt.subplot(2,2,3)
    plt.imshow(mask,cmap="gray")
    plt.title("Generated mask based on treshold | Binary")
    plt.axis("off")
    
    
    plt.subplot(2,2,4)
    plt.imshow(final,cmap="gray")
    plt.axis("off")
    plt.title("Final image")
    
    plt.tight_layout()
    plt.show()
    
    return final  

# P2: Iterative Algorithm

In [ ]:
def iterative(img_gray):
    start = time.time()
    img_flat = img_gray.flatten()
    p_i = np.array([(img_flat==level).sum() for level in range(256)],dtype=np.float64)
    p_i /= img_flat.shape
    
    h,w = img_gray.shape
    img_flat = img_gray.flatten()
    corners = [(0,0),(0,w-1),(h-1,0),(h-1,w-1)]
    back_pix = []
    for_pix = []
    for row in range(h):
        for col in range(w):
            if (row,col) in corners :
                back_pix.append(img_gray[row,col])
            else:
                for_pix.append(img_gray[row,col])
            
    for_tresh = np.mean(for_pix)
    back_tresh = np.mean(back_pix)
    new_tresh = np.mean([back_tresh,for_tresh])
    tresh = -1
    ite = 0
    treshs = []
    while new_tresh != tresh:
        treshs.append(new_tresh)
        tresh = new_tresh
        back_pix = img_flat[np.where(img_flat<= tresh)]
        for_pix = img_flat[np.where(img_flat> tresh)]
        
        for_tresh = np.mean(for_pix)
        back_tresh = np.mean(back_pix)
        new_tresh = np.mean([back_tresh,for_tresh])
        
        ite += 1 
        
    ret,mask = cv2.threshold(img_gray,tresh,255,cv2.THRESH_BINARY)
    final = cv2.bitwise_and(img_gray,mask)
    end = round( time.time()-start,2)
    tresh = round(tresh,2)
    
    plt.subplot(2,2,1)
    plt.imshow(img_gray,cmap="gray")
    plt.title("Input image")
    plt.axis("off")
    
    
    plt.subplot(2,2,2)
    plt.bar(list(range(256)),p_i,color="tab:cyan")
    
    plt.scatter(treshs,[np.mean(p_i)]*len(treshs),color="blue",alpha=0.6,s=30)
    
    plt.plot([tresh,tresh],[0,max(p_i)],color="red")
    plt.axis("off")
    plt.title("Iterative algorithm | iterate:{0} | Threshold:{1} | Time:{2}".format(len(treshs),tresh,end))
    
    
    plt.subplot(2,2,3)
    plt.imshow(mask,cmap="gray")
    plt.title("Generated mask based on treshold | Binary")
    plt.axis("off")
    
    
    plt.subplot(2,2,4)
    plt.imshow(final,cmap="gray")
    plt.axis("off")
    plt.title("Final image")
    
    plt.tight_layout()
    plt.show()
    return final    

    


# Apply OTSU and iterative on test images

In [ ]:
test_1 = cv2.imread("./test-2-noise.jpg",0)
test_2 = cv2.imread("./test-7.png",0)
test_3 = cv2.imread("./test-13.png",0)
test_4 = cv2.imread("./test-2-noise.jpg",0)

#otsu(test_1)
#iterative(test_1)
#otsu(test_2)
#iterative(test_2)


# P4: Template Matching

In [ ]:
sns.set_style("dark")
def templateMatching(img_gray:np.ndarray,template_gray:np.ndarray,to_rotate=False):
    if not to_rotate:
        matched_img = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)
        t_h,t_w = template_gray.shape
        matched_loc = np.where(matched_img>=.99)
        matched_loc = np.array(list(zip(matched_loc[1],matched_loc[0])))
        for matched in matched_loc:
            matched_loc_top_left = matched
            matched_loc_bot_right = (matched_loc_top_left[0] + t_w, matched_loc_top_left[1] + t_h)
            cv2.rectangle(img_gray,matched_loc_top_left, matched_loc_bot_right, 255, 2)
            
        plt.rcParams['figure.figsize'] = (16, 8)
        plt.subplot(1,2,1)
        plt.imshow(matched_img,cmap="gray")
        plt.title("prob. map | matched image")
        plt.axis("off")
        
        plt.subplot(1,2,2)
        plt.imshow(img_gray,cmap="gray")
        plt.title("Detected Cameras")
        plt.axis("off")
        plt.show()
            
        return 
    
    best_matches = []
    a,b = 15,375
    for angle in range(a,b):
        temp_r = rotate_image(template_gray,angle)
        matched_img = cv2.matchTemplate(img_gray,temp_r,cv2.TM_CCOEFF_NORMED)
        best_matches.append(np.max(matched_img.flatten()))
    best_matches = np.array(best_matches)
    
    found_angels = argrelextrema(best_matches, np.greater,order=10)[0]
    
    peaks_x = []
    peaks_y = []
    plt.rcParams['figure.figsize'] = (5,15)
    for angle in found_angels:
        temp_r = rotate_image(template_gray,angle)  
        matched_img = cv2.matchTemplate(img_gray,temp_r,cv2.TM_CCOEFF_NORMED)
        _, max_val, ___,matched_loc = cv2.minMaxLoc(matched_img)
        
        peaks_x.append(angle+15)
        peaks_y.append(best_matches[angle])
        t_h,t_w = temp_r.shape
        matched_loc_top_left = matched_loc
        matched_loc_bot_right = (matched_loc_top_left[0] + t_w, matched_loc_top_left[1] + t_h)
        cv2.rectangle(img_gray,matched_loc_top_left, matched_loc_bot_right, (255), 5)

    

    plt.rcParams['figure.figsize'] = (16, 8)
    plt.subplot(1,2,1)
    plt.plot(list(range(a,b)),best_matches)
    plt.scatter(peaks_x,peaks_y,color="green")   
    for i in range(len(peaks_x)):
        plt.text(peaks_x[i]+1,peaks_y[i]+0.003,str(peaks_x[i]),color="red")
    plt.title("max probability to rotate`s angle signal")
    
    plt.subplot(1,2,2)
    plt.imshow(img_gray,cmap="gray")
    plt.title("Detected Cameras")
    plt.axis("off")
    plt.show()

in_img = cv2.imread("./img.jpeg",0)
tmp_img = cv2.imread("./template.jpeg",0)
templateMatching(in_img.copy(),tmp_img.copy(),False)
templateMatching(in_img.copy(),tmp_img.copy(),True)

    